In [ ]:
!pip install neo4j

In [ ]:
import requests
from neo4j import GraphDatabase

In [ ]:
url = 'neo4j://localhost:7687'
driver = GraphDatabase.driver(url, auth=('neo4j', 'changeme'))
driver.verify_connectivity()

In [ ]:
import_url = 'https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/e90b0c9386bf8be15b199e8ac8f83fc265a2ac57/microservices.json'
import_query = requests.get(import_url).json()['query']
with driver.session(database='neo4j') as session:
    session.run(import_query)
    session.close()

In [ ]:
r = driver.execute_query('MATCH (n:Person) RETURN n LIMIT 25')
for rec in r.records:
    print(rec["n"]["name"])

In [ ]:
index_name='tasks'
node_label="Task"
text_node_properties=['name', 'description', 'status']
embedding_node_property='embedding'
embedding_dimension=1536
similarity_metric='cosine'

In [ ]:
retrieval_query = (
                f"RETURN reduce(str='', k IN {text_node_properties} |"
                " str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, "
                "node {.*, `"
                + embedding_node_property
                + "`: Null, id: Null, "
                + ", ".join([f"`{prop}`: Null" for prop in text_node_properties])
                + "} AS metadata, score"
)

In [ ]:
retrieval_query

In [ ]:
r = driver.execute_query(
    "SHOW INDEXES YIELD name, type, labelsOrTypes, properties, options "
    "WHERE type = 'VECTOR' AND (name = $index_name "
    "OR (labelsOrTypes[0] = $node_label AND "
    "properties[0] = $embedding_node_property)) "
    "RETURN name, labelsOrTypes, properties, options ",
    {
        "index_name": index_name,
        "node_label": node_label,
        "embedding_node_property": embedding_node_property,
    }
)

In [ ]:
r.records

In [ ]:
r = driver.execute_query(
    "CALL db.index.vector.createNodeIndex("
    "$index_name,"
    "$node_label,"
    "$embedding_node_property,"
    "toInteger($embedding_dimension),"
    "$similarity_metric )",
    {
        "index_name": index_name,
        "node_label": node_label,
        "embedding_node_property": embedding_node_property,
        "embedding_dimension": embedding_dimension,
        "similarity_metric": similarity_metric,
    }
)

In [ ]:
r.summary

In [ ]:
fetch_query = (
                f"MATCH (n:`{node_label}`) "
                f"WHERE n.{embedding_node_property} IS null "
                "AND any(k in $props WHERE n[k] IS NOT null) "
                f"RETURN elementId(n) AS id, reduce(str='',"
                "k IN $props | str + '\\n' + k + ':' + coalesce(n[k], '')) AS text "
                "LIMIT 1000"
            )

In [ ]:
fetch_query

In [ ]:
r = driver.execute_query(fetch_query, { "props": text_node_properties })

In [ ]:
r.records

In [ ]:
for row in r.records:
    print(row["text"])